In [1]:
import pyaudio
import wave
import sys
import os

# DOES NOT RUN ON UBUNTU 20.04 / PYTHON 3.9 BECAUSE I CAN'T GET pyaudiIo TO INSTALL PROPERLY 

On my Ubuntu 20.04 machine I tried to install pyaudio using
```
sudo apt-get install -y python3-pyaudio
```
FAILED

In [2]:
sys.version

'3.7.3 (default, Jan 22 2021, 20:04:44) \n[GCC 8.3.0]'

In [11]:
p = pyaudio.PyAudio()
for dev_index in range(p.get_device_count()):
    print(dev_index, p.get_device_info_by_index(dev_index).get('name'))

0 bcm2835 HDMI 1: - (hw:0,0)
1 Cable Creation: USB Audio (hw:1,0)
2 bcm2835 Headphones: - (hw:2,0)
3 sysdefault
4 lavrate
5 samplerate
6 speexrate
7 pulse
8 upmix
9 vdownmix
10 dmix
11 default


In [3]:
# These parameters may be set by papermill
AUDIO_FORMAT = pyaudio.paInt16    # 16-bit resolution
CHANNELS = 1                      # 1 channel
RATE = 44100                      # 44.1kHz sampling rate
FRAMES_PER_BUFFER = 4096          # 2^12 samples for buffer
RECORD_SECS = 60                  # seconds to record
DEV_INDEX = 1                    # device index found by p.get_device_info_by_index()
WAV_OUTPUT_FILENAME = 'lunga_up.wav' # name of .wav file

In [13]:
def record():
    audio = pyaudio.PyAudio() # create pyaudio instantiation

    # create pyaudio stream
    stream = audio.open(format = AUDIO_FORMAT,
                        rate = RATE,
                        channels = CHANNELS,
                        input_device_index = DEV_INDEX,
                        input = True,
                        frames_per_buffer = FRAMES_PER_BUFFER)
    print("recording")
    frames = []

    # loop through stream and append audio chunks to frame array
    for chunk in range(0,int((RATE/FRAMES_PER_BUFFER)*RECORD_SECS)):
        data = stream.read(FRAMES_PER_BUFFER)
        frames.append(data)

    print("finished recording")

    # stop the stream, close it, and terminate the pyaudio instantiation
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # save the audio frames as .wav file
    wavefile = wave.open(WAV_OUTPUT_FILENAME,'wb')
    wavefile.setnchannels(CHANNELS)
    wavefile.setsampwidth(audio.get_sample_size(AUDIO_FORMAT))
    wavefile.setframerate(RATE)
    wavefile.writeframes(b''.join(frames))
    wavefile.close()

In [7]:
%%time

def update_github():
    os.system("git pull")
    os.system("git add .")
    os.system("git commit -m 'commited by record_wav.ipynb'")
    os.system("git push")
update_github()

Already up to date.
[master 5868e7d] commited by record_wav.ipynb
 2 files changed, 120 insertions(+), 32 deletions(-)
CPU times: user 31 ms, sys: 20.9 ms, total: 52 ms
Wall time: 4.19 s


fatal: could not read Username for 'https://github.com': No such device or address


'.'

In [14]:
# for i in range(11):
#     WAV_OUTPUT_FILENAME = f'(i}.wav')
#     record()

In [15]:
# MAIN

record()

recording
finished recording
